In [1]:
import os
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from mmseg.apis import init_segmentor, inference_segmentor
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# Set up models

In [2]:
config_file = '../cache/upernet_swin_base_patch4_window7_512x512_80k_FTU_whole_multilabel/upernet_swin_base_patch4_window7_512x512_80k_FTU_whole_multilabel.py'
checkpoint_file = '../cache/upernet_swin_base_patch4_window7_512x512_80k_FTU_whole_multilabel/best_mDice_iter_40000.pth'
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/zhangzr/mmsegmentation_kaggle/mmseg/models/losses/cross_entropy_loss.py:242: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: ../cache/upernet_swin_base_patch4_window7_512x512_80k_FTU_whole_multilabel/best_mDice_iter_40000.pth


In [3]:
def rle_encode(mask: np.ndarray, label_bg: int = 0) -> Dict[int, str]:
    """Encode mask to Run-length encoding.
    Inspiration took from: https://gist.github.com/nvictus/66627b580c13068589957d6ab0919e66
    >>> from pprint import pprint
    >>> mask = np.array([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    ...                  [0, 0, 0, 1, 1, 1, 2, 2, 2, 0],
    ...                  [0, 0, 0, 0, 0, 1, 3, 3, 0, 0],])
    >>> pprint(rle_encode(mask))
    {1: '1 5 13 3 25 1', 2: '16 3', 3: '26 2'}
    """
    vec = mask.flatten()
    nb = len(vec)
    where = np.flatnonzero
    starts = np.r_[0, where(~np.isclose(vec[1:], vec[:-1], equal_nan=True)) + 1]
    lengths = np.diff(np.r_[starts, nb])
    values = vec[starts]
    assert len(starts) == len(lengths) == len(values)
    rle = {}
    for start, length, val in zip(starts, lengths, values):
        if val == label_bg:
            continue
        rle[val] = rle.get(val, []) + [str(start), length]
    # post-processing
    rle = {lb: " ".join(map(str, id_lens)) for lb, id_lens in rle.items()}
    return rle

# Make test submission csv

In [4]:
data_dir = "../data/hubmap-organ-segmentation"
test_dir = os.path.join(data_dir, "test_images")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.tiff"), recursive = True)
sub

,id,rle
0,10078,12 34


# predict

In [5]:
def maxLabel(mask):
    unique, counts = np.unique(mask, return_counts=True)
    index = np.argmax(counts[1:]) + 1
    return unique[index]
    

In [11]:
for index,img_name in enumerate(test_images):
    id = img_name.split('/')[-1].split('.')[0]
    res = inference_segmentor(model, img_name)[0]
    label = maxLabel(res)
    rle = rle_encode(res.T)
    sub.loc[index, "id"] = id
    sub.loc[index, "rle"] = rle[label]


# Format submission

In [7]:
sub.to_csv("submission.csv", index = False)
sub

,id,rle
0,10078,853553 5 855573 10 857592 15 859595 37 861614 ...


In [9]:
print(rle.keys())

dict_keys([4, 1])
